# Performing Testing at Each Round of Training 

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-24 14:34:32,118 fedbiomed INFO - Component environment:
2022-03-24 14:34:32,119 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-24 14:34:32,275 fedbiomed INFO - Messaging researcher_eb917e3f-7dc3-4cf7-aeb5-8dff56d304f0 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f228701fa00>
2022-03-24 14:34:32,337 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-24 14:34:42,348 fedbiomed INFO - Node selected for training -> node_30468209-f851-49b3-8c65-1d7c22f71078
2022-03-24 14:34:42,370 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0007/my_model_03311a7c-0779-40a0-9920-6b1060df8915.py
2022-03-24 14:34:42,387 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0007/my_model_03311a7c-0779-40a0-9920-6b1060df8915.py successful, with status code 201
2022-03-24 14:34:42,528 fedbi

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [4]:
exp.run()

2022-03-24 14:34:42,541 fedbiomed INFO - Sampled nodes in round 0 ['node_30468209-f851-49b3-8c65-1d7c22f71078']
2022-03-24 14:34:42,543 fedbiomed INFO - Send message to node node_30468209-f851-49b3-8c65-1d7c22f71078 - {'researcher_id': 'researcher_eb917e3f-7dc3-4cf7-aeb5-8dff56d304f0', 'job_id': 'eeb16596-f47d-4c14-8cc9-c127c2eacc15', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/24/my_model_03311a7c-0779-40a0-9920-6b1060df8915.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/24/aggregated_params_init_5042d843-3276-4c3f-b54d-551fcbd5cd03.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_30468209-f851-49b3-8c65-1d7c22f71078': ['dataset_5bb7413d-873d-4a52-8cbc-56270474b

2022-03-24 14:35:17,632 fedbiomed INFO - TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.266156 
					 ---------
2022-03-24 14:35:18,137 fedbiomed INFO - TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.146449 
					 ---------
2022-03-24 14:35:24,920 fedbiomed INFO - TESTING BEFORE TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Completed: 18000/18000 (100%) 
 					 ACCURACY: 0.958611 
					 ---------
2022-03-24 14:35:25,049 fedbiomed INFO - INFO FROM NODE node_30468209-f851-49b3-8c65-1d7c22f71078
MESSAGE: 
results uploaded successfully 
----------------------------------------
2022-03-24 14:35:32,783 fedbiomed INFO - Downloading model params after training on node_30468209-f851-49b3-8c65-1d7c22f71078 - from http://localhost:8844/media/uploads/2022/03/24/node_params_48078064-4857-4b58-b0d3-9da020ce3059.

2

In [5]:
exp.run(rounds=8, increase=True)

2022-03-23 14:05:50,215 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 10
2022-03-23 14:05:50,219 fedbiomed INFO - Sampled nodes in round 2 ['node_bc74a204-a162-4b7d-8629-06dca3148c4b']
2022-03-23 14:05:50,221 fedbiomed INFO - Send message to node node_bc74a204-a162-4b7d-8629-06dca3148c4b - {'researcher_id': 'researcher_18aa50a8-9be3-4c41-94cc-123916147007', 'job_id': 'e94f8ffe-06cf-4e79-8369-833afd03363b', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/23/my_model_6504001b-e590-4f91-890e-3c3d971b0dc5.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/23/aggregated_params_49b759f2-5c5d-4f5c-ae36-158abfffc758.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bc74a204-a162-4b7d-8629-06dca3148c4b': ['dataset_e9c5a4b6-6516-4494-9a12-543538e55399']}}
2022-03-23 14:05:50,222 fedbiomed DEBUG - r

2022-03-23 14:06:25,664 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 3840/48000 (8%) 
 					 Loss: 0.018571 
					 ---------
2022-03-23 14:06:26,207 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 4320/48000 (9%) 
 					 Loss: 0.078840 
					 ---------
2022-03-23 14:06:31,454 fedbiomed INFO - TESTING AFTER TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Completed: 12000/12000 (100%) 
 					 ACCURACY: 0.974750 
					 ---------
2022-03-23 14:06:31,610 fedbiomed INFO - log from: node_bc74a204-a162-4b7d-8629-06dca3148c4b / INFO - results uploaded successfully 
2022-03-23 14:06:40,488 fedbiomed INFO - Downloading model params after training on node_bc74a204-a162-4b7d-8629-06dca3148c4b - from http://localhost:8844/media/uploads/2022/03/23/node_params_2f371643-aa2e-4e3e-88ae-4e2af1049a96.pt
2022-03-23 14:06:40,532 fedbiomed DEBUG - uplo

2022-03-23 14:07:12,101 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 480/48000 (1%) 
 					 Loss: 0.308031 
					 ---------
2022-03-23 14:07:12,632 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 960/48000 (2%) 
 					 Loss: 0.047183 
					 ---------
2022-03-23 14:07:13,154 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 1440/48000 (3%) 
 					 Loss: 0.048069 
					 ---------
2022-03-23 14:07:13,686 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 1920/48000 (4%) 
 					 Loss: 0.080780 
					 ---------
2022-03-23 14:07:14,213 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 2400/48000 (5%) 
 					 Loss: 0.148342 
					 ---------
2022-03-23 14:07:14,739 fedbiomed INF

2022-03-23 14:07:56,433 fedbiomed INFO - Send message to node node_bc74a204-a162-4b7d-8629-06dca3148c4b - {'researcher_id': 'researcher_18aa50a8-9be3-4c41-94cc-123916147007', 'job_id': 'e94f8ffe-06cf-4e79-8369-833afd03363b', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/23/my_model_6504001b-e590-4f91-890e-3c3d971b0dc5.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/23/aggregated_params_7a9e24f5-029b-411f-9e2f-769474cc3218.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bc74a204-a162-4b7d-8629-06dca3148c4b': ['dataset_e9c5a4b6-6516-4494-9a12-543538e55399']}}
2022-03-23 14:07:56,433 fedbiomed DEBUG - researcher_18aa50a8-9be3-4c41-94cc-123916147007
2022-03-23 14:07:56,520 fedbiomed INFO - log from: node_bc74a204-a162-4b7d-8629-06dca3148c4b / INFO - training with arguments {'history_monitor': <fedbiomed.node.hi

2022-03-23 14:08:32,269 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Epoch: 1 | Completed: 4320/48000 (9%) 
 					 Loss: 0.107408 
					 ---------
2022-03-23 14:08:37,619 fedbiomed INFO - TESTING AFTER TRAINING 
					 NODE_ID: node_bc74a204-a162-4b7d-8629-06dca3148c4b 
					 Completed: 12000/12000 (100%) 
 					 ACCURACY: 0.983917 
					 ---------
2022-03-23 14:08:37,767 fedbiomed INFO - log from: node_bc74a204-a162-4b7d-8629-06dca3148c4b / INFO - results uploaded successfully 
2022-03-23 14:08:46,716 fedbiomed INFO - Downloading model params after training on node_bc74a204-a162-4b7d-8629-06dca3148c4b - from http://localhost:8844/media/uploads/2022/03/23/node_params_748eaf8a-6a45-4e87-a4e2-e758f527ef8e.pt
2022-03-23 14:08:46,764 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_c3b77fa5-48c8-4ac6-a882-98a91cbf955e.pt successful, with status code 200
2022-03-23 14:08:46,775 fedbiomed INFO - Nodes that successfully reply in roun

8

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [5]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()


List the training rounds :  dict_keys([0, 1])

List the nodes for the last training round and their timings : 
	- node_bc74a204-a162-4b7d-8629-06dca3148c4b :    
		rtime_training=5.50 seconds    
		ptime_training=21.83 seconds    
		rtime_total=25.03 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_e9c5a4b6-6516-4494-9a12-543538e55399,node_bc74a204-a162-4b7d-8629-06dca3148c4b,/home/ybouilla/fedbiomed/var/experiments/Exper...,"{'conv1.weight': [[tensor([[-0.2764, 0.3102, ...","{'rtime_training': 5.504039911000291, 'ptime_t..."


Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [6]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1])

Access the federated params for the last training round :
	- params_path:  /home/ybouilla/fedbiomed/var/experiments/Experiment_0017/aggregated_params_73630e50-0bb0-4831-b89d-d6b900a5b8e3.pt
	- parameter data:  odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


Feel free to run other sample notebooks or try your own models :D

## Testing using your own testing metric

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlanCM(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlanCM, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

    def testing_step(self, data, target):
        
        output = self.forward(data)
        loss1   = torch.nn.functional.nll_loss(output, target)
        output = self(data)
        loss2   = torch.nn.functional.nll_loss(output, target)
        return [loss1, loss2]

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlanCM,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-24 14:47:07,749 fedbiomed INFO - Component environment:
2022-03-24 14:47:07,750 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-24 14:47:07,903 fedbiomed INFO - Messaging researcher_eb917e3f-7dc3-4cf7-aeb5-8dff56d304f0 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7faf15a57e50>
2022-03-24 14:47:07,961 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-24 14:47:17,972 fedbiomed INFO - Node selected for training -> node_30468209-f851-49b3-8c65-1d7c22f71078
2022-03-24 14:47:18,012 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0009/my_model_adcac230-c80a-4f3e-bfd4-5f7ceeab9ecd.py
2022-03-24 14:47:18,028 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0009/my_model_adcac230-c80a-4f3e-bfd4-5f7ceeab9ecd.py successful, with status code 201
2022-03-24 14:47:18,168 fedbi

In [4]:
exp.run()

2022-03-24 14:47:18,173 fedbiomed INFO - Sampled nodes in round 0 ['node_30468209-f851-49b3-8c65-1d7c22f71078']
2022-03-24 14:47:18,173 fedbiomed INFO - Send message to node node_30468209-f851-49b3-8c65-1d7c22f71078 - {'researcher_id': 'researcher_eb917e3f-7dc3-4cf7-aeb5-8dff56d304f0', 'job_id': '4c5ee515-c575-4863-8f81-9f3b87727cc1', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/24/my_model_adcac230-c80a-4f3e-bfd4-5f7ceeab9ecd.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/24/aggregated_params_init_ba5758fd-8a70-46d2-8ca1-0df9cdc19308.pt', 'model_class': 'MyTrainingPlanCM', 'training_data': {'node_30468209-f851-49b3-8c65-1d7c22f71078': ['dataset_5bb7413d-873d-4a52-8cbc-5627047

2022-03-24 14:48:09,507 fedbiomed INFO - TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.356098 
					 ---------
2022-03-24 14:48:10,001 fedbiomed INFO - TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.224149 
					 ---------
2022-03-24 14:48:22,227 fedbiomed INFO - TESTING BEFORE TRAINING 
					 NODE_ID: node_30468209-f851-49b3-8c65-1d7c22f71078 
					 Completed: 18000/18000 (100%) 
 					 Custom_1: 0.127800 
					 Custom_2: 0.127800 
					 ---------
2022-03-24 14:48:22,403 fedbiomed INFO - INFO FROM NODE node_30468209-f851-49b3-8c65-1d7c22f71078
MESSAGE: 
results uploaded successfully 
----------------------------------------
2022-03-24 14:48:28,430 fedbiomed INFO - Downloading model params after training on node_30468209-f851-49b3-8c65-1d7c22f71078 - from http://localhost:8844/media/uploads/2022/03/24/node_params_b869c7ee-38

2